Before running this notebook, please run

```bash
s2gos-server run --service=s2gos_server.services.airflow.testing:service
```

In [ ]:
from s2gos_client import Client
from s2gos_common.models import ProcessRequest

In [ ]:
client = Client(server_url="http://127.0.0.1:8008")
client

In [ ]:
client.get_capabilities()

In [ ]:
client.get_conformance()

In [ ]:
client.get_processes()

In [ ]:
client.get_process(process_id="sleep_a_while")

In [ ]:
client.get_jobs()

In [ ]:
client.execute_process(process_id="sleep_a_while", request=ProcessRequest())

In [ ]:
client.execute_process(process_id="sleep_a_while", request={})

In [ ]:
client.execute_process(process_id="primes_between", request={})

In [ ]:
client.get_jobs()

In [ ]:
# client.get_job_results("job_1")

In [ ]:
for job in client.get_jobs().jobs:
    client.dismiss_job(job.jobID)

In [ ]:
client.get_jobs()